# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Data Loading & Data understanding

In [2]:
df_movie = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
df_keyword = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')

/tmp/ipykernel_13/2197234160.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')


## Data Movies

Pada Dataset Movie ini memiliki sekitar 45 ribu baris data (masih memiliki missing value dan duplicate data) dan 24 kolom :
- adult: Menunjukkan apakah film ditujukan untuk penonton dewasa atau tidak.
- belongs_to_collection: Berisi informasi tentang apakah film tersebut merupakan bagian dari sebuah koleksi atau waralaba (franchise). 
- budget: Anggaran produksi film dalam bentuk angka, tetapi disimpan sebagai string. 
- genres: Berisi informasi mengenai genre film dalam format JSON. 
- homepage: URL dari situs web resmi film, jika ada.
- id: ID unik untuk setiap film di dataset.
- imdb_id: ID film yang terdaftar di situs IMDb, yang dapat digunakan untuk pencarian di platform tersebut.
- original_language: Bahasa asli dari film.
- original_title: Judul asli film dalam bahasa aslinya, sebelum diterjemahkan atau diubah untuk pasar internasional.
- overview: Sinopsis singkat atau ringkasan dari plot film.
- popularity: Ukuran popularitas film di berbagai platform. 
- poster_path: Path ke poster film yang dapat digunakan untuk menampilkan gambar poster melalui situs TMDb.
- production_companies: Berisi nama perusahaan produksi film dalam format JSON.
- production_countries: Negara tempat film diproduksi, disimpan dalam format JSON.
- release_date: Tanggal rilis film. Biasanya dalam format YYYY-MM-DD.
- revenue: Pendapatan global yang dihasilkan oleh film, dalam dolar AS.
- runtime: Durasi film dalam menit.
- spoken_languages: Bahasa yang digunakan dalam film, disimpan dalam format JSON.
- status: Status produksi film.
- tagline: Slogan atau tagline dari film, yang sering kali digunakan dalam promosi.
- title: Judul resmi film yang mungkin berbeda dari judul aslinya (original_title).
- video: Menunjukkan apakah film tersebut memiliki trailer video (True atau False).
- vote_average: Skor rata-rata film berdasarkan penilaian dari pengguna.
- vote_count: Jumlah total suara atau penilaian yang diterima oleh film.

In [3]:
df_movie.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
# Mendeteksi Duplikat Data
df_movie.duplicated().sum()

13

In [6]:
# Memastikan data tidak ada yang missing value
df_movie.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

Seperti yang kita lihat bahwa hampir seluruh kolom memiliki missing value. Ada banyak cara dalam menangani missing value, namun pada kasus kali ini kita akan menghapus beberapa kolom karena tidak terlalu berpengaruh pada rekomendasi movie, untuk itu hanya kolom `original_title`, `genres`, `vote_average`, `overview` dan `Id` yang akan digunakan pada df_movie yang nantinya akan digabungkan dengan data keywords

In [7]:
df_movies = df_movie[['id', 'original_title', 'genres', 'overview', 'vote_average']]
# Membuat index pada dataset
df_movies['index'] = df_movies.index
df_movies.head()

/tmp/ipykernel_13/328680658.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['index'] = df_movies.index


,id,original_title,genres,overview,vote_average,index
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",7.7,0
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,6.9,1
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,6.5,2
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",6.1,3
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,5.7,4


In [8]:
print(f"the number of columns of the datasets: {df_movies.shape[1]}")
print(f"the number of rows of the datasets: {df_movies.shape[0]}")

the number of columns of the datasets: 6
the number of rows of the datasets: 45466


In [9]:
# Menghapus baris di mana id tidak ada atau jika bukan angka
df_movies = df_movies[df_movies['id'].str.isnumeric()]

In [10]:
print(f"the number of columns of the datasets: {df_movies.shape[1]}")
print(f"the number of rows of the datasets: {df_movies.shape[0]}")

the number of columns of the datasets: 6
the number of rows of the datasets: 45463


## Data Keywords

Pada Dataset Movie ini memiliki sekitar 46 ribu baris data (masih memiliki missing value dan duplicate data) dan 2 kolom :
- id: ID unik untuk setiap film, yang sesuai dengan kolom id di dataset movie. Ini adalah kunci yang dapat digunakan untuk menghubungkan kedua dataset.
- keywords: Berisi kata kunci atau tag yang relevan dengan film, dalam format JSON. Kata kunci ini menggambarkan tema atau elemen utama dalam film.

In [11]:
df_keyword.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [12]:
df_keyword.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [13]:
print(f"the number of columns of the datasets: {df_keyword.shape[1]}")
print(f"the number of rows of the datasets: {df_keyword.shape[0]}")

the number of columns of the datasets: 2
the number of rows of the datasets: 46419


In [14]:
# Mendeteksi Duplikat Data
df_keyword.duplicated().sum()

987

In [15]:
# Hapus duplikasi data
df_keyword = df_keyword.drop_duplicates()
# Cek duplikat data setelah duplikat dihapus
df_keyword.duplicated().sum()

0

In [16]:
print(f"the number of columns of the datasets: {df_keyword.shape[1]}")
print(f"the number of rows of the datasets: {df_keyword.shape[0]}")

the number of columns of the datasets: 2
the number of rows of the datasets: 45432


In [17]:
# Memastikan data tidak ada yang missing value
df_keyword.isnull().sum()

id          0
keywords    0
dtype: int64

# Data Preprocessing

## Data Merge

Pada tahapan ini kita melakukan merge data movies dengan data keywords

In [18]:
df_movies['id'] = df_movies['id'].astype(int) # Ubah tipe data ke int karna sebelumnya adalah object
df_movies_all = pd.merge(df_movies, df_keyword, on='id', how='left')
df_movies_all.reset_index(inplace=True, drop=True)
df_movies_all.head()

,id,original_title,genres,overview,vote_average,index,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",7.7,0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,6.9,1,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,6.5,2,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",6.1,3,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,5.7,4,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


Setelah dilakukan merge data movie dan data keyword berikut adalah informasi dari `data_movies_all` yang nantinya data ini akan digunakan untuk dilakukan pemodelan:
- id: ID unik untuk setiap film di dataset.
- original_title: Judul asli film dalam bahasa aslinya, sebelum diterjemahkan atau diubah untuk pasar internasional.
- genres: Berisi informasi mengenai genre film dalam format JSON.
- vote_average: Skor rata-rata film berdasarkan penilaian dari pengguna.
- overview: Sinopsis singkat atau ringkasan dari plot film.
- keywords: Berisi kata kunci atau tag yang relevan dengan film, dalam format JSON. Kata kunci ini menggambarkan tema atau elemen utama dalam film.


In [19]:
print(f"the number of columns of the datasets: {df_movies_all.shape[1]}")
print(f"the number of rows of the datasets: {df_movies_all.shape[0]}")

the number of columns of the datasets: 7
the number of rows of the datasets: 45463


Dikarenakan data `df_movies_all` ini hasil dari penggabungan dari kedua dataset untuk itu dilakukan pemeriksaan apakah data memiliki missing value atau duplikasi pada data, jika terdapat missing value atau duplikasi data kita hapus agar akurasi model saat merekomendasikan film bagus dan sesuai dengan preferensi model

In [20]:
# Memastikan data tidak duplikat
df_movies_all.duplicated().sum()

0

In [21]:
# Hapus Duplikasi data
df_movies_all = df_movies_all.drop_duplicates()
# Cek duplikat data setelah duplikat dihapus
df_movies_all.duplicated().sum()

0

In [22]:
# Memastikan data tidak ada yang missing value
df_movies_all.isnull().sum()

id                  0
original_title      0
genres              0
overview          954
vote_average        3
index               0
keywords            1
dtype: int64

In [23]:
# Hapus missing value pada data
df_movies_all = df_movies_all.dropna()
# Cek missing value setelah dihapus
df_movies_all.isnull().sum()

id                0
original_title    0
genres            0
overview          0
vote_average      0
index             0
keywords          0
dtype: int64

In [24]:
print(f"the number of columns of the datasets: {df_movies_all.shape[1]}")
print(f"the number of rows of the datasets: {df_movies_all.shape[0]}")

the number of columns of the datasets: 7
the number of rows of the datasets: 44505


Seperti yang dilihat diawal pada `df_movies_all.head()` isi dari kolom `genres` dan `keywords` itu memiliki banyak value (valuenya masih berformat JSON) untuk itu kita harus membersihkan dan mangambil nilai yang ingin yang kita pakai

In [25]:
# Ubah kolom genres yang hanya menampilkan genre saja tanpa id
df_movies_all['genres'] = df_movies_all['genres'].apply(lambda x: [i['name'] for i in eval(x)])
df_movies_all.head()

,id,original_title,genres,overview,vote_average,index,keywords
0,862,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",7.7,0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,6.9,1,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,6.5,2,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",6.1,3,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,5.7,4,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [26]:
# Menghapus list pada kolom genre
df_movies_all['genres'] = df_movies_all['genres'].apply(lambda x: ' '.join(x))
df_movies_all.head()

,id,original_title,genres,overview,vote_average,index,keywords
0,862,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",7.7,0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...,6.9,1,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,Romance Comedy,A family wedding reignites the ancient feud be...,6.5,2,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom...",6.1,3,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,Comedy,Just when George Banks has recovered from his ...,5.7,4,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [27]:
# Melakukan hal yang sama pada kolom keywords
df_movies_all['keywords'] = df_movies_all['keywords'].apply(lambda x: [i['name'] for i in eval(x)])
df_movies_all['keywords'] = df_movies_all['keywords'].apply(lambda x: ' '.join([i.replace(" ", "") for i in x]))
# Hapus kolom id
df_movies_all.drop(['id'], axis=1, inplace=True)
df_movies_all.head()

,original_title,genres,overview,vote_average,index,keywords
0,Toy Story,Animation Comedy Family,"Led by Woody, Andy's toys live happily in his ...",7.7,0,jealousy toy boy friendship friends rivalry bo...
1,Jumanji,Adventure Fantasy Family,When siblings Judy and Peter discover an encha...,6.9,1,boardgame disappearance basedonchildren'sbook ...
2,Grumpier Old Men,Romance Comedy,A family wedding reignites the ancient feud be...,6.5,2,fishing bestfriend duringcreditsstinger oldmen
3,Waiting to Exhale,Comedy Drama Romance,"Cheated on, mistreated and stepped on, the wom...",6.1,3,basedonnovel interracialrelationship singlemot...
4,Father of the Bride Part II,Comedy,Just when George Banks has recovered from his ...,5.7,4,baby midlifecrisis confidence aging daughter m...


Membuat kolom baru bernama `tags` dengan melakukan kombinasi dari kolom `original_title`, `genres` dan `keywords` untuk dikenversikan ke bentuk vectors menggunakan TF-IDF

In [28]:
#combine all selected features
df_movies_all['tags'] = df_movies_all["original_title"]+" "+df_movies_all["genres"]+" "+df_movies_all["keywords"]
df_movies_all.drop(['keywords'], axis=1, inplace=True)
df_movies_all = df_movies_all[['index', 'original_title', 'genres', 'vote_average', 'tags']]
df_movies_all.head()

,index,original_title,genres,vote_average,tags
0,0,Toy Story,Animation Comedy Family,7.7,Toy Story Animation Comedy Family jealousy toy...
1,1,Jumanji,Adventure Fantasy Family,6.9,Jumanji Adventure Fantasy Family boardgame dis...
2,2,Grumpier Old Men,Romance Comedy,6.5,Grumpier Old Men Romance Comedy fishing bestfr...
3,3,Waiting to Exhale,Comedy Drama Romance,6.1,Waiting to Exhale Comedy Drama Romance basedon...
4,4,Father of the Bride Part II,Comedy,5.7,Father of the Bride Part II Comedy baby midlif...


# Modeling

Untuk model kali ini kita akan menggunakan Content Based Filtering dimana tujuan dari dibuatnya model ini adalah mencari kesamaan antara film dengan menggunakan cosine similarity, sehingga menampilkan preferensi sesuai dengan yang user cari atau ketertarikan user menyukai movie tersebut.

Sebelum kita membuat model kita harus ubah kolom `tags` ke bentuk vectors menggunakan teknik TF-IDF, Metode ini mengubah informasi tekstual menjadi nilai numerik, yang kemudian dapat digunakan untuk menghitung kemiripan (menggunakan cosine similarity) antar film.

In [29]:
# Konversi data text ke vectors
vectorizer = TfidfVectorizer()
feature_vektor = vectorizer.fit_transform(df_movies_all['tags'])

# Lalu hitung cosine similarity dari hasil vectors dari matrix TF-IDF
similarity = cosine_similarity(feature_vektor)

melakukan pemeriksaan ukuran dari fitur yang telah divektorkan dan melakukan pemeriksaan ukuran dari hasil hitung cosine similarity

In [30]:
# Cek ukuran dari feature_vektor
print(feature_vektor.shape)

# Cek ukuran dari df_movies_all['original_title']
print(similarity.shape)


(44505, 48325)
(44505, 44505)


Buat dataframe untuk melihat kemungkinan kemiripan antar film dari hasil perhitungan cosine similarity

In [31]:
# Membuat dataframe untuk melihat hasil dari cosine similarity
df_similarity = pd.DataFrame(similarity, columns=df_movies_all['original_title'], 
             index=df_movies_all['original_title']).sample(10, axis=1).sample(10, axis=0)
df_similarity

original_title,Balto II: Wolf Quest,George of the Jungle 2,Maybe Baby,The Reader,Le jour et la nuit,The Great Raid,All The Queen's Men,Cet Amour-là,Nine Muses of Star Empire,The Decline of Western Civilization Part III
original_title,,,,,,,,,,
Half Moon Street,0.000000,0.000000,0.000000,0.004345,0.007185,0.000000,0.007441,0.007341,0.000000,0.000000
11 Blocks,0.000000,0.000000,0.000000,0.000000,0.000000,0.013770,0.023601,0.000000,0.000000,0.000000
Logan's Run,0.020286,0.028221,0.013590,0.008307,0.000000,0.008489,0.014550,0.000000,0.000000,0.000000
Queen Sized,0.056025,0.077938,0.000000,0.008232,0.013613,0.000000,0.231491,0.013909,0.000000,0.000000
Crocodile,0.000000,0.000000,0.000000,0.000000,0.000000,0.008304,0.014233,0.000000,0.000000,0.000000
Asylum of Darkness,0.000000,0.057242,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039335,0.038599
Kisna: The Warrior Poet,0.000000,0.030431,0.028115,0.037385,0.011078,0.013489,0.034592,0.011319,0.000000,0.020519
Smokey and the Bandit Part 3,0.000000,0.054693,0.017312,0.013684,0.000000,0.031473,0.072063,0.000000,0.000000,0.191417
Kater,0.000000,0.000000,0.000000,0.004006,0.006624,0.000000,0.006860,0.006769,0.000000,0.000000


Setelah itu buat fungsi untuk menampilkan rekomendasi dari preferensi user, dengan proses sistem rekomendasi sebagai berikut:
1. Pengguna mencari atau memasukkan judul film.
2. Sistem menggunakan metode `difflib.get_close_matches` untuk menemukan judul yang paling dekat dengan set data.
3. Setelah film yang benar diidentifikasi, sistem menghitung kemiripan antara film yang dipilih dan semua film lainnya menggunakan matriks cosine similarity.
4. Sistem mengembalikan daftar film teratas yang paling mirip dengan film yang dipilih berdasarkan skor cosine similarity.

In [32]:
def recommend_movie(movie_name, df_movies_all, similarity_matrix, k=15): 
    """
    Merekomendasikan film berdasarkan kemiripan judul dengan dataset yang ada.
    
    Parameters:
    - movie_name (str): Nama film yang diinput oleh pengguna
    - df_movies_all (DataFrame): Dataset yang berisi informasi film, termasuk kolom 'original_title', 'genre', dan 'vote_average'
    - similarity_matrix (array): Matriks kemiripan antar film
    - k (int): Jumlah rekomendasi yang diinginkan (default: 10)
    
    Returns:
    - DataFrame: Rekomendasi film dalam bentuk DataFrame
    """
    # Mengambil daftar semua judul film dari dataset
    list_of_all_titles = df_movies_all["original_title"].tolist()
    
    # Mencari judul yang paling mirip dengan input pengguna
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    
    if find_close_match:
        # Jika ditemukan film yang mirip, gunakan yang pertama dalam daftar
        close_match = find_close_match[0]
        
        # Mendapatkan indeks dari film yang cocok
        matched_movie = df_movies_all[df_movies_all['original_title'] == close_match]
        
        if not matched_movie.empty:  # Cek apakah film ditemukan
            index_of_the_movie = matched_movie["index"].values[0]
            
            # Mendapatkan skor kemiripan dengan film lainnya berdasarkan similarity_matrix
            similarity_scores = list(enumerate(similarity_matrix[index_of_the_movie]))
            
            # Mengurutkan film berdasarkan skor kemiripan secara menurun
            sorted_similarity_movie = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
            
            # Menyiapkan list untuk menyimpan rekomendasi film
            recommendations = []
            
            # Menampilkan hingga k film yang paling mirip, kecuali film itu sendiri (indeks 0)
            for movie in sorted_similarity_movie[1:k+1]:  # Mulai dari indeks 1, lewati film itu sendiri
                index = movie[0]
                
                # Ambil judul, genre, dan vote_average
                title = df_movies_all.iloc[index]['original_title']
                genre = df_movies_all.iloc[index]['genres']
                vote_average = df_movies_all.iloc[index]['vote_average']
                
                # Tambahkan ke list rekomendasi
                recommendations.append({
                    'Title': title,
                    'Genre': genre,
                    'Vote Average': vote_average
                })
                
            # Mengembalikan rekomendasi dalam bentuk DataFrame
            return pd.DataFrame(recommendations)
        else:
            print(f"Sorry, the movie '{close_match}' was not found in the dataset.")
            return None
    else:
        print(f"Sorry, no close match found for '{movie_name}'.")
        return None

Selanjutnya kita dapat mencoba menggunakan fungsi tersebut untuk menampilkan film yang direkomendasikan sesuai dengan pencarian atau ketertarikan user terhadap film

In [33]:
movie_name = input("Movie apa yang ingin kamu tonton: ")

# Memanggil fungsi untuk merekomendasikan film
recommendation_movie = recommend_movie(movie_name, df_movies_all, similarity)
recommendation_movie

Movie apa yang ingin kamu tonton:  The Matrix


,Title,Genre,Vote Average
0,Zero,Drama,8.0
1,Road to Morocco,Comedy,7.4
2,Morocco,Drama Romance,6.6
3,Brief Interviews with Hideous Men,Comedy Drama Romance,5.1
4,Arabian Adventure,Fantasy Science Fiction Family,5.8
5,Arabian Nights,Drama Family Fantasy,6.9
6,A Sister's Revenge,Thriller Drama Mystery,3.3
7,The Hideous Sun Demon,Horror Science Fiction,5.7
8,Little Sister,Comedy,4.0
9,Little Sister,Comedy,6.2


# Evaluasi

Dari model yang telah dibuat kita akan evaluasi apakah hasilnya cukup bagus atau tidak, dimana kita menggunakan precision sebagai evaluasi model yang kita telah buat

In [34]:
k = 15
threshold = 5
movie_ratings = recommendation_movie['Vote Average'].values
movie_relevances = movie_ratings > threshold
precision = len(movie_ratings[movie_relevances]) / k
print(f'The precision of the recommendation system is {precision:.1%}')

The precision of the recommendation system is 73.3%


Dengan hasil precision 73,3%, model ini sudah menjawab problem statement dengan cukup baik. Precision mengukur seberapa akurat rekomendasi yang diberikan model, yakni proporsi film yang direkomendasikan dan disukai pengguna dari semua yang disarankan. Nilai 73,3% menunjukkan bahwa sekitar 3 dari 4 film yang direkomendasikan sesuai dengan preferensi pengguna. Artinya, model berhasil memberikan rekomendasi yang relevan dan berkualitas, menjawab kebutuhan untuk memberikan saran yang personal.